# Скачать датасет с фразами Симпсонов

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('simpsons_script_lines.csv', on_bad_lines='skip')

<ipython-input-143-103a6cb3f8b9>:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('simpsons_script_lines.csv', on_bad_lines='skip')


In [ ]:
df.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31.0
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3.0
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22.0
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5.0
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40.0,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33.0


# Обучить word2vec на фразах персонажей

Модель Word2Vec — это алгоритм для обучения векторных представлений слов на основе их контекста. Эта модель является одной из самых популярных и эффективных для обработки естественного языка. Word2Vec принадлежит к классу нейронных сетей прямого распространения (feed-forward neural networks) и обучается на больших текстовых корпусах.

Существует два основных варианта модели Word2Vec: Continuous Bag of Words (CBOW) и Skip-gram. В CBOW модели контекстные слова подаются на вход, а целевое слово предсказывается. В Skip-gram модели наоборот, целевое слово подается на вход, а контекстные слова предсказываются. Оба варианта метода используются для обучения векторных представлений слов.

После обучения модели Word2Vec каждому слову сопоставляется вектор фиксированной размерности. Эти векторы представляют слова и заносят в себя семантическую и синтаксическую информацию о словах. Векторы представлений слов в модели Word2Vec обладают интересными свойствами, такими как близость слов с похожими значениями в пространстве и возможность выполнения алгебраических операций с ними.

## Предобработка текста

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
sentences = [word_tokenize(str(text)) for text in df['normalized_text']]

In [ ]:
sentences[:5]

[['no',
  'actually',
  'it',
  'was',
  'a',
  'little',
  'of',
  'both',
  'sometimes',
  'when',
  'a',
  'disease',
  'is',
  'in',
  'all',
  'the',
  'magazines',
  'and',
  'all',
  'the',
  'news',
  'shows',
  'its',
  'only',
  'natural',
  'that',
  'you',
  'think',
  'you',
  'have',
  'it'],
 ['wheres', 'mr', 'bergstrom'],
 ['i',
  'dont',
  'know',
  'although',
  'id',
  'sure',
  'like',
  'to',
  'talk',
  'to',
  'him',
  'he',
  'didnt',
  'touch',
  'my',
  'lesson',
  'plan',
  'what',
  'did',
  'he',
  'teach',
  'you'],
 ['that', 'life', 'is', 'worth', 'living'],
 ['the',
  'polls',
  'will',
  'be',
  'open',
  'from',
  'now',
  'until',
  'the',
  'end',
  'of',
  'recess',
  'now',
  'just',
  'in',
  'case',
  'any',
  'of',
  'you',
  'have',
  'decided',
  'to',
  'put',
  'any',
  'thought',
  'into',
  'this',
  'well',
  'have',
  'our',
  'final',
  'statements',
  'martin']]

## Обучение модели

In [ ]:
!pip install gensim

In [ ]:
from gensim import models

In [ ]:
model = models.Word2Vec()
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(4854736, 6689390)

# Визуализировать embeddings по самым частотным словам (top 1000)

**t-SNE**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).

In [ ]:
# импортируем необходимые библиотеки
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE
import numpy as np

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
top_words = []

# строим простой словарь частотности употреблений всех слов
fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

#оставляем только 1000 самых частых слов
for w in fd.most_common(1000):
    top_words.append(w[0])

print(top_words[:50:])

<ipython-input-153-edc2c7136b8e>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm(sentences):


  0%|          | 0/158248 [00:00<?, ?it/s]

['the', 'you', 'i', 'a', 'to', 'nan', 'and', 'of', 'it', 'that', 'my', 'in', 'is', 'this', 'me', 'your', 'for', 'im', 'we', 'on', 'oh', 'what', 'have', 'no', 'but', 'be', 'well', 'its', 'dont', 'all', 'just', 'with', 'are', 'do', 'not', 'now', 'like', 'so', 'was', 'get', 'can', 'youre', 'know', 'one', 'got', 'at', '--', 'thats', 'up', 'out']


In [ ]:
top_words_vec = [model.wv[word] for word in top_words]

In [ ]:
top_words_vec = np.array(top_words_vec)

In [ ]:
%%time
# инициализируем модель
tsne = TSNE(n_components=2, random_state=0)
# обучаем и применяем
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 30 s, sys: 119 ms, total: 30.1 s
Wall time: 29.4 s


In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (Simpsons dataset, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

# Найти самые близкие слова для: homer - marge + bart bart - lisa + school marge - homer + home

In [ ]:
result_homer_marge_bart = model.wv.most_similar(positive=["homer", "bart"], negative=["marge"], topn=5)

In [ ]:
print("Words similar to 'homer - marge + bart':", result_homer_marge_bart)

Words similar to 'homer - marge + bart': [('lisa', 0.7835456728935242), ('grampa', 0.6969323754310608), ('mrs', 0.6303368806838989), ('milhouse', 0.6195242404937744), ('abe', 0.601315438747406)]


In [ ]:
result_bart_lisa_school = model.wv.most_similar(positive=["bart", "school"], negative=["lisa"], topn=5)

In [ ]:
print("Words similar to 'bart - lisa + school':", result_bart_lisa_school)

Words similar to 'bart - lisa + school': [('church', 0.6951183080673218), ('plant', 0.6663619875907898), ('house', 0.6620638966560364), ('class', 0.6565156579017639), ('store', 0.6552029252052307)]


In [ ]:
result_marge_homer_home = model.wv.most_similar(positive=["marge", "home"], negative=["homer"], topn=5)

In [ ]:
print("Words similar to 'marge - homer + home':", result_marge_homer_home)

Words similar to 'marge - homer + home': [('back', 0.723633348941803), ('sleep', 0.6611122488975525), ('bed', 0.6581592559814453), ('boys', 0.5809769630432129), ('here', 0.5439883470535278)]
